In [4]:
import pandas as pd

va_filtered_listings = pd.read_csv("va_filtered_listings.csv")

# Display the first few rows
va_filtered_listings.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,5/1/2019,7/31/2019,90,zip code,2,9670,f,Zip Code: 24244,NaN,Virginia,...,0.100000,NaN,NaN,NaN,0.100000,0.000000,0.100000,"Kingsport, TN",28700,2/10/2025 14:21
1,6/1/2022,8/31/2022,90,zip code,2,9223,f,Zip Code: 23235,NaN,Virginia,...,0.037794,NaN,NaN,NaN,0.680272,-0.059373,-0.023216,"Richmond, VA",40060,2/10/2025 14:21
2,9/1/2024,11/30/2024,90,zip code,2,9824,f,Zip Code: 24563,NaN,Virginia,...,-0.716667,NaN,NaN,NaN,0.375000,-0.069444,0.000000,"Danville, VA",19260,2/10/2025 14:21
3,8/1/2018,10/31/2018,90,zip code,2,7894,f,Zip Code: 20187,NaN,Virginia,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,"Washington, DC",47894,2/10/2025 14:21
4,11/1/2022,1/31/2023,90,zip code,2,9224,f,Zip Code: 23236,NaN,Virginia,...,-0.084934,NaN,NaN,NaN,0.452830,-0.032464,-0.188679,"Richmond, VA",40060,2/10/2025 14:21


#### There is no data for the city column, so we have to use the zip codes to pick specific counties for our research. We will draw in a zip code dataset for this

In [5]:
# Get zip code county data
geo_data_url = 'https://raw.githubusercontent.com/scpike/us-state-county-zip/master/geo-data.csv'
df_geo = pd.read_csv(geo_data_url)

print('Num of rows: ', len(df_geo))
print('Num of columns: ', len(df_geo.columns))
df_geo.head()

Num of rows:  33103
Num of columns:  6


,state_fips,state,state_abbr,zipcode,county,city
0,1,Alabama,AL,35004,St. Clair,Acmar
1,1,Alabama,AL,35005,Jefferson,Adamsville
2,1,Alabama,AL,35006,Jefferson,Adger
3,1,Alabama,AL,35007,Shelby,Keystone
4,1,Alabama,AL,35010,Tallapoosa,New site


In [7]:
# Filter on a single county and create a copy
df_geo_county = df_geo.loc[(df_geo['state_abbr'] == 'VA') &
                           (df_geo['county'].isin(['Albemarle']))].copy()

# Create a feature to check if zip code is valid
df_geo_county['valid_zip_code'] = df_geo_county['zipcode'].astype(str).str.isnumeric()

# Display the dataset summary
print('Dataset: ')
print(df_geo_county.groupby(['valid_zip_code', 'county'])['zipcode'].count())
print(' ')
print('Invalid zip codes: ', df_geo_county.loc[df_geo_county['valid_zip_code'] == False]['zipcode'].unique())


Dataset: 
valid_zip_code  county   
False           Albemarle     2
True            Albemarle    14
Name: zipcode, dtype: int64
 
Invalid zip codes:  ['229HH' '245HH']


In [9]:
# Keep only valid zip codes
df_geo_county_valid = df_geo_county[df_geo_county['valid_zip_code'] == True]
print('Num of VALID zip codes: ', len(df_geo_county_valid))
df_geo_county_valid.head()

# Extract zip code from 'region' field in df_filter
df_filter['zipcode'] = df_filter['region'].apply(lambda x: x.split(':')[1].strip())

# Merge market data and geographic zip codes
df_merge = pd.merge(
    df_filter,
    df_geo_county_valid[['zipcode', 'city', 'county']],  # Keep relevant columns
    how='inner',  # Only keep matching zip codes from both tables
    on=['zipcode']  # Merge on zip code column
)

# Display results
print('Num of rows after merging: ', len(df_merge))
df_merge.head()


Num of VALID zip codes:  14
Num of rows after merging:  4428


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city_x,state,...,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated,zipcode,city_y,county
0,6/1/2022,8/31/2022,90,zip code,2,9054,f,Zip Code: 22932,NaN,Virginia,...,NaN,0.650000,-0.115957,0.650000,"Charlottesville, VA",16820,2/10/2025 14:21,22932,Yancey mills,Albemarle
1,6/1/2019,8/31/2019,90,zip code,2,9048,f,Zip Code: 22911,NaN,Virginia,...,NaN,0.012987,0.007369,0.006364,"Charlottesville, VA",16820,2/10/2025 14:21,22911,Zcta 22911,Albemarle
2,10/1/2024,12/31/2024,90,zip code,2,9054,f,Zip Code: 22932,NaN,Virginia,...,NaN,0.310345,0.000000,-0.097063,"Charlottesville, VA",16820,2/10/2025 14:21,22932,Yancey mills,Albemarle
3,3/1/2020,5/31/2020,90,zip code,2,9065,f,Zip Code: 22947,NaN,Virginia,...,NaN,0.000000,0.000000,0.000000,"Charlottesville, VA",16820,2/10/2025 14:21,22947,Boyd tavern,Albemarle
4,2/1/2018,4/30/2018,90,zip code,2,9841,f,Zip Code: 24590,NaN,Virginia,...,NaN,NaN,NaN,NaN,"Charlottesville, VA",16820,2/10/2025 14:21,24590,Scottsville,Albemarle


In [10]:
# Save the merged dataset as a new CSV file
df_merge.to_csv("va_filtered_listings_albemarle.csv", index=False)